In [1]:
from __future__ import division, absolute_import

import os
import sys
import random

import numpy as np

#root
absPath = '/home/angela/padding_EBI/'
sys.path.insert(0, absPath)

#from src.Target import Target

np.random.seed(8)
random.seed(8)

from src.preprocessing import *
from src.model_architecture import *
from src.training_model import *
from src.postprocessing import *
from src.comparing_results import *
#from src.callbacks import *
%matplotlib inline

Using TensorFlow backend.


### Defining parameters

In [2]:
list_paddings = ['post_padding', 'pre_padding', 'mid_padding', 'stretch_padding', 'ext_padding', 
                 'rdm_padding', 'aug_padding']
#list_padding_short = ['post_padding', 'pre_padding', 'mid_padding', 'stretch_padding', 'ext_padding', 
#                 'rdm_padding']
#hierarchy of folders: annotation/dataset/architecture/n_neurs/task/padding
folder = 'EC_number/archaea/3denses/bio_neurons/'
task = "task2/"
column = "EC number"

n_folds = 10

dicti = creating_dict()

max_lenn = 1000

n_class = 2 #number of classes to output
drop_per = 0.2 #Input dropout 
n_neur = [314,77, 8]
drop_hid = 0.5
dict_size = len(dicti)

batch_size = 54
epochss = 200

### Loading stuff

In [3]:
file_idcs_aug = os.path.join(absPath, 'data/', folder, 'idcs_aug_enz.pickle')

with open(file_idcs_aug, "rb") as input_file:
    k_aug_indices = pickle.load(input_file)
    
file_idcs = os.path.join(absPath, 'data/', folder, 'idcs_data_enz.pickle')

with open(file_idcs, "rb") as input_file:
    splitting_sets = pickle.load(input_file)

In [4]:
for model_type in list_paddings:
    if model_type == "aug_padding":
        indices = splitting_sets
    else:
        indices = k_aug_indices
    for idx,i in enumerate(indices):
        print('Fold: ', idx)
        i_train, i_val, i_test = i
        print(i_test)
        #loading best model
        his_folder = ''.join(string for string in [absPath, 'data/results/', folder, task, model_type, 
                                                   '/', str(idx)])
        history = plot_history(his_folder)
        path_to_cp = ''.join(string for string in [absPath, 'data/checkpoint/', folder, task, model_type, 
                                            '/', str(idx)])
        model, best_path = load_best_model(history, path_to_cp)
        cps_loc = ''.join(string for string in [absPath, 'data/checkpoint/', folder, task, model_type, 
                                            '/', str(idx), '/*.hdf5'])
        #removing the rest of weights
        fileList = glob.glob(cps_loc, recursive=True)
        fileList.remove(best_path)
        if len(fileList) >1:
            for filePath in fileList:
                try:
                    os.remove(filePath)
                except OSError:
                    print("Error while deleting file")
        #loading test data        
        if model_type == "aug_padding":
            file_data = os.path.join(absPath, 'data/', folder, 'aug_data.h5')
        else:
            file_data = os.path.join(absPath, 'data/', folder, 'data.h5')
        h5f = h5py.File(file_data, 'r')
    
        instarget = Target('AAAAAA')
        x_test = h5f[model_type][sorted(i_test)]
        x_test = instarget.int_to_onehot(list(x_test), len(dicti))
        y_test = h5f["labels_task2"][sorted(i_test)]
    
        #predicting 
        y_predprob = model.predict(x_test)
        y_pred = y_predprob.argmax(axis=-1)
        y_test_scalar = y_test.argmax(axis=-1)
        y_prob = y_predprob[:,1]
    
        print(Counter(y_pred))
    
        #confusion matrix
        #file_conf = ''.join(string for string in [his_folder, '/resulting_metrics.pickle'])    
        confusion_matrix(y_test_scalar, y_pred, his_folder)
    
        #AUC
        #file_auc = ''.join(string for string in [his_folder, '/AUC.pickle'])
        compute_roc(y_test_scalar, y_prob, his_folder )
    
        computing_partial_auc(y_test_scalar, y_prob, his_folder)

Fold:  0
[36785, 465, 52123, 54062, 56637, 48805, 59409, 55135, 87394, 43074, 2140, 31369, 56639, 37766, 25642, 47630, 111480, 69234, 39857, 43078, 104306, 34743, 86992, 22920, 35486, 108847, 105850, 62170, 47250, 104783, 37762, 108019, 7736, 80940, 37729, 110858, 51418, 108674, 46788, 36510, 105248, 34425, 92757, 16858, 32243, 63698, 35561, 11711, 61196, 51213, 70312, 83513, 71413, 1795, 66492, 108870, 4107, 104304, 35557, 87495, 93168, 107607, 110756, 80059, 55817, 108871, 52348, 94380, 46652, 91937, 96117, 105796, 23444, 66808, 7717, 52310, 11369, 52309, 33990, 49032, 47502, 116004, 108618, 22821, 36879, 48030, 13289, 113380, 66497, 59877, 62869, 111099, 12947, 86956, 37730, 7737, 14065, 80941, 67556, 96399, 50100, 48807, 39944, 48377, 33197, 96400, 75197, 94089, 66561, 75025, 111485, 32231, 86766, 110171, 22461, 69773, 52126, 39942, 47621, 15479, 105675, 86767, 68914, 107509, 6255, 98869, 59127, 4516, 63489, 107565, 31848, 48237, 39928, 97540, 108706, 111853, 11542, 22148, 43289, 9

FileNotFoundError: [Errno 2] No such file or directory: '/home/angela/padding_EBI/data/results/EC_number/archaea/3denses/bio_neurons/task2/post_padding/0/history.pickle'